## Integer Linear Programming:  Set-Covering Problems

In [5]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable
import pandas as pd
import numpy as np

The greater metropolitan area of Easton, IL includes 17 suburbs that are serviced by the Easton Fire and Rescue Department.  Because of the proximities of some of the suburbs, one suburb's fire station may serve neighboring communities.  The requirement is that the station must be closer than a 15 minute drive from any suburb that it serves.  The data file FireStations_DriveTimes.csv contains the drive times in minutes between the 17 suburbs.

Formulate an ILP to identify the smallest number of stations that will cover the area.  Where are they located?



In [6]:
data = pd.read_csv('FireStations_DriveTimes.csv')
data = data.set_index('Suburb')
data

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q
Suburb,,,,,,,,,,,,,,,,,
A,0,37,58,17,20,8,31,52,56,13,33,16,12,10,34,15,26
B,37,0,54,15,15,39,26,28,12,25,40,33,50,12,25,12,27
C,58,54,0,58,59,49,17,30,50,22,12,47,44,13,54,23,19
D,17,15,58,0,42,31,35,44,55,31,40,21,59,47,35,30,51
E,20,15,59,42,0,18,29,35,48,30,36,18,60,46,43,45,15
F,8,39,49,31,18,0,25,42,15,30,23,56,43,54,40,59,25
G,31,26,17,35,29,25,0,16,36,8,53,44,15,59,46,40,16
H,52,28,30,44,35,42,16,0,29,24,14,26,35,16,50,47,36
I,56,12,50,55,48,15,36,29,0,31,57,13,18,15,59,40,58


In [7]:
min_time = 15
data_matrix = data.applymap(lambda x: 1 if x < min_time else 0)
data_matrix

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q
Suburb,,,,,,,,,,,,,,,,,
A,1,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0
B,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0
C,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0
D,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
E,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
F,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
H,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
I,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0


In [8]:
model = LpProblem("Fire_Station_Location", LpMinimize)

In [9]:
# Construct Decision Variables
locations = LpVariable.dicts("station", (data.index), cat = 'Binary')
locations

{'A': station_A,
 'B': station_B,
 'C': station_C,
 'D': station_D,
 'E': station_E,
 'F': station_F,
 'G': station_G,
 'H': station_H,
 'I': station_I,
 'J': station_J,
 'K': station_K,
 'L': station_L,
 'M': station_M,
 'N': station_N,
 'O': station_O,
 'P': station_P,
 'Q': station_Q}

In [10]:
# Add the objective function to the model

In [17]:
# Add the constraints to the model

for s in data.index:
    model += (lpSum(locations[x] * data_matrix.loc[s,x] for x in data.index) >=  1)

In [18]:
model

Fire_Station_Location:
MINIMIZE
None
SUBJECT TO
_C1: station_A + station_F + station_J + station_M + station_N >= 1

_C2: station_B + station_I + station_N + station_P >= 1

_C3: station_C + station_K + station_N >= 1

_C4: station_D >= 1

_C5: station_E >= 1

_C6: station_A + station_F >= 1

_C7: station_G + station_J >= 1

_C8: station_H + station_K >= 1

_C9: station_B + station_I + station_L >= 1

_C10: station_A + station_G + station_J >= 1

_C11: station_C + station_H + station_K + station_Q >= 1

_C12: station_I + station_L + station_O >= 1

_C13: station_A + station_M >= 1

_C14: station_A + station_B + station_C + station_N >= 1

_C15: station_L + station_O >= 1

_C16: station_B + station_P >= 1

_C17: station_K + station_Q >= 1

VARIABLES
__dummy = 0 Continuous
0 <= station_A <= 1 Integer
0 <= station_B <= 1 Integer
0 <= station_C <= 1 Integer
0 <= station_D <= 1 Integer
0 <= station_E <= 1 Integer
0 <= station_F <= 1 Integer
0 <= station_G <= 1 Integer
0 <= station_H <= 1 In

In [19]:
model.solve()

1

In [20]:
model.objective.value()

AttributeError: 'NoneType' object has no attribute 'value'

In [ ]:
for v in model.variables(): 
    if v.varValue == 1 :
        print(f"{v.name}: {v.varValue}")